# Prediction example for Hainan Gibbons (25 samples) while freezing the feature extractor

**Paper:** Passive Acoustic Monitoring and Transfer Learning

**Authors:** Emmanuel Dufourq, Carly Batist, Ruben Foquet and Ian Durbach

**Repository:** https://github.com/emmanueldufourq/PAM_TransferLearning

<hr>

**Note 1: Set colab=True if executing on Google Colab, or set colab=False if running locally.**

**Note 2: When running on Google Colab, please select Runtime > Change runtime type > GPU**

<hr>

## Step 0: Download data and code

In [ ]:
colab = True

In [ ]:
if colab:
    !pip install SoundFile
    from google.colab import drive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    import zipfile

    # Google Authentication
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    # Download model files
    downloaded = drive.CreateFile({'id':"11KD0g40pyN5eirLHRtnD9MPQnKqxCy_0"})
    downloaded.GetContentFile('2564512-ResNet152V2_NoFineTuned_Gibbons.hdf5')

    # Download data files
    downloaded = drive.CreateFile({'id':"1ikTmmCos2by8VEmfrAxvEoqRAoE_9JcK"})
    downloaded.GetContentFile('Data.zip')

    # Extract files to temporary location in Google Drive
    with zipfile.ZipFile('Data.zip', 'r') as zip_file:
        zip_file.extractall()

    # Download code files
    downloaded = drive.CreateFile({'id':"11zmF1E521uugpI9BIiYLDJRi6O7JxXUM"})
    downloaded.GetContentFile('Code.zip')

    # Extract files to temporary location in Google Drive
    with zipfile.ZipFile('Code.zip', 'r') as zip_file:
        zip_file.extractall()

In [ ]:
!pip install yattag
from PredictionHelper import *

In [ ]:
species_folder = 'Data' # Should contain /Audio and /Annotations
lowpass_cutoff = 2000 # Cutt off for low pass filter
downsample_rate = 4800 # Frequency to downsample to
nyquist_rate = 2400 # Nyquist rate (half of sampling rate)
segment_duration = 4 # how long should a segment be
augmentation_amount_positive_class = 1 # how many times should a segment be augmented
augmentation_amount_negative_class = 1 # how many times should a segment be augmented
positive_class = ['gibbon'] # which labels should be bundled together for the positive  class
negative_class = ['no-gibbon'] # which labels should be bundled together for the negative  class
n_fft = 1024 # Hann window length
hop_length = 256 # Sepctrogram hop size
n_mels = 128 # Spectrogram number of mells
f_min = 4000 # Spectrogram, minimum frequency for call
f_max = 9000 # Spectrogram, maximum frequency for call
weights_name = '2564512-ResNet152V2_NoFineTuned_Gibbons.hdf5' # Name of the TF NN weights to load

## Step 1: Predict on each testing file

In [ ]:
predict = Prediction(species_folder, lowpass_cutoff, 
                downsample_rate, nyquist_rate, 
                segment_duration,
                positive_class, negative_class, 
                augmentation_amount_positive_class, 
                augmentation_amount_negative_class,n_fft, 
                hop_length, n_mels, f_min, f_max, weights_name)

In [ ]:
predict.predict_all_test_files(True)

## Step 2: Download .svl files from ```Data/Model_Output/``` and import into SonicVisualiser to visualise the predictions
